In [1]:
using CSV
using DataFrames
using Serialization
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/md_data_2020_12_01.csv", missingstring="null"));

In [4]:
hospitals = sort(unique(rawdata[!,"Facility Name"]));
get_hospital_id(h) = h |> hash |> string |> (x -> "h" * x[end-7:end]);
convert_date(d) = Date(d, dateformat"m/d/y") + Year(2000);

data = select(rawdata,
    "Facility Name" => :hospital,
    "Facility Name" => ByRow(get_hospital_id) => :hospital_id,

    "Occupied Adult Acute Care Beds" => :acute_occupancy,
    "Occupied Adult ICU Beds" => :icu_occupancy,
    ["Occupied Adult Acute Care Beds", "Occupied Adult ICU Beds"] => ByRow((a,b) -> a+b) => :total_occupancy,
);
sort!(data, :hospital);

In [5]:
function latest_val(xs)
    xs = filter(x -> !ismissing(x), xs)
    z = length(xs) == 0 ? missing : xs[end]
    z = coalesce(z, 0)
    return z
end;

In [6]:
data_latest = combine(groupby(data, :hospital), [
    :hospital_id => latest_val => :hospital_id,
    :total_occupancy => latest_val => :total_occupancy,
    :icu_occupancy => latest_val => :icu_occupancy,
    :acute_occupancy => latest_val => :acute_occupancy,
]);

In [7]:
capacity_rawdata = DataFrame(CSV.File("../data/capacity.csv"));

In [8]:
capacity_data = select(capacity_rawdata,
    :hospital,
    :capacity_icu => :icu_beds,
    :capacity_acute => :acute_beds,
    :capacity_allbeds => :total_beds,
);

In [9]:
data_latest = rightjoin(data_latest, capacity_data, on=:hospital);

In [10]:
compute_load(a,b) = (a==0) ? 0.0 : (b==0) ? 1.0 : a/b;

In [11]:
data_latest.total_load = compute_load.(data_latest.total_occupancy, data_latest.total_beds);
data_latest.icu_load = compute_load.(data_latest.icu_occupancy, data_latest.icu_beds);
data_latest.acute_load = compute_load.(data_latest.acute_occupancy, data_latest.acute_beds);

In [12]:
metadata = DataFrame(CSV.File("../data/hospital_meta.csv"));

In [13]:
data_combined = leftjoin(data_latest, metadata, on=:hospital_id, makeunique=true);

In [14]:
sort!(data_combined, :hospital);

In [15]:
data_combined

,hospital,hospital_id,total_occupancy,icu_occupancy,acute_occupancy,icu_beds,acute_beds,total_beds,total_load,icu_load,acute_load,hospital_1,in_forecast,address,county,healthcare_region,city,zipcode,lat,long,place_id,state,state_abbrev,hsa_id,hsa_name,hrr_id,hrr_name
,String?,String?,Int64?,Int64?,Int64?,Int64,Int64,Int64,Float64,Float64,Float64,String?,Bool?,String?,String?,String?,String?,Int64?,Float64?,Float64?,String?,String?,String?,Int64?,String?,Int64?,String?
1,Anne Arundel Medical Center,h39740093,301,27,274,40,284,324,0.929012,0.675,0.964789,Anne Arundel Medical Center,1,"2002 Medical Pkwy, Annapolis, MD 21401, USA",Anne Arundel County,Region III,Annapolis,21401,38.9904,-76.5374,ChIJwQbdhNf2t4kR9wSFLYAT0Go,Maryland,MD,21001,"Annapolis, MD",113,"Washington, DC"
2,Atlantic General Hospital,h92616877,44,7,37,9,43,51,0.862745,0.777778,0.860465,Atlantic General Hospital,1,"9733 Healthway Dr, Berlin, MD 21811, USA",Worcester County,Region IV,Berlin,21811,38.3396,-75.2115,ChIJrTSS5gInuYkRtT2krn-1-Jw,Maryland,MD,21029,"Salisbury, MD",225,"Salisbury, MD"
3,Baltimore Convention Center Field Hospital,h94720559,32,0,32,0,26,23,1.3913,0.0,1.23077,Baltimore Convention Center Field Hospital,0,"1 W Pratt St, Baltimore, MD 21201, USA",Baltimore City,Region III,Baltimore,21201,39.2854,-76.6171,ChIJqSGk9WADyIkRDVkLuWmZKdM,Maryland,MD,21002,"Baltimore, MD",223,"Baltimore, MD"
4,Baltimore Washington Medical Center,h75227743,233,32,201,30,206,236,0.987288,1.06667,0.975728,Baltimore Washington Medical Center,1,"301 Hospital Dr, Glen Burnie, MD 21061, USA",Anne Arundel County,Region III,Glen Burnie,21061,39.1379,-76.6218,ChIJofV1-bz8t4kRUG1GNyt_7cA,Maryland,MD,21016,"Glen Burnie, MD",223,"Baltimore, MD"
5,Bowie Medical Center,h26261837,0,0,0,0,10,10,0.0,0.0,0.0,Bowie Medical Center,0,"15001 Health Center Dr, Bowie, MD 20716, USA",Prince Georges County,Region V,Bowie,20716,38.9494,-76.746,ChIJn0x8bI7rt4kR2KeO6HUDCGY,Maryland,MD,21020,"Lanham, MD",226,"Takoma Park, MD"
6,Calvert Memorial Hospital,h51968182,48,6,42,6,54,59,0.813559,1.0,0.777778,Calvert Memorial Hospital,1,"100 Hospital Rd, Prince Frederick, MD 20678, USA",Calvert County,Region V,Prince Frederick,20678,38.5594,-76.5963,ChIJGf93tYmCt4kRUHcfYxctVoM,Maryland,MD,21025,"Prince Frederick, MD",113,"Washington, DC"
7,Carroll Hospital Center,h73144964,129,7,122,10,112,123,1.04878,0.7,1.08929,Carroll Hospital Center,1,"200 Memorial Ave, Westminster, MD 21157, USA",Carroll County,Region III,Westminster,21157,39.5576,-76.9903,ChIJX6DoR2s4yIkR_eqhMva6JnY,Maryland,MD,21033,"Westminster, MD",223,"Baltimore, MD"
8,Charles Regional (UM),h61059656,74,8,66,12,98,111,0.666667,0.666667,0.673469,Charles Regional (UM),1,"5 Garrett Ave, La Plata, MD 20646, USA",Charles County,Region V,La Plata,20646,38.529,-76.9722,ChIJL0cYbucIt4kR8Bh1L4nlJtc,Maryland,MD,21019,"La Plata, MD",113,"Washington, DC"
9,Chestertown (UMSRH),h95946725,10,1,9,2,11,13,0.769231,0.5,0.818182,Chestertown (UMSRH),1,"223 High St, Chestertown, MD 21620, USA",Kent County,Region IV,Chestertown,21620,39.2085,-76.0657,ChIJ_1Qaz0OLx4kRb8E80Y10zW0,Maryland,MD,21005,"Chestertown, MD",223,"Baltimore, MD"


In [16]:
data_combined |> CSV.write("../data/current_load.csv");

In [17]:
data_combined_list = collect([NamedTuple(h) for h in eachrow(data_combined)]);

In [18]:
serialize("../data/current_load.jlser", data_combined_list)